In [14]:

import numpy as np
from qiskit.algorithms import QAOA
from qiskit.algorithms.optimizers import COBYLA
from qiskit.opflow import PauliSumOp
from qiskit import Aer, QuantumCircuit, execute
import time
import warnings

warnings.filterwarnings("ignore", category=DeprecationWarning)

# ================== 系统参数配置 ==================
n = 2  # 作业数量
m = 3  # 机器数量
l = 2  # C_max二进制位数
N = int(np.ceil(np.log2(m)))  # 机器编码位数
p = np.array([[1, 2, 3], [2, 3, 1]])  # 处理时间矩阵

valid_machines = {0: [0, 0], 1: [0, 1], 2: [1, 0]}
illegal_machines = [[1, 1]]
num_qubits = n * N + l


# ================== 哈密顿量构建 ==================
def build_hamiltonian():
    Q = np.zeros((num_qubits, num_qubits))
    K1, P = 1000, 2000

    # 1. 非法分配惩罚项
    for illegal in illegal_machines:
        for i in range(n):
            terms = machine_indicator(i * N, illegal)
            for (q, r), coeff in terms.items():
                Q[q, r] += K1 * coeff

    # 2. 时间约束项
    for mid, bits in valid_machines.items():
        for i in range(n):
            terms = machine_indicator(i * N, bits)
            p_ij = p[i][mid]

            # 处理时间平方项
            for (q, r), coeff in terms.items():
                Q[q, r] += P * (p_ij ** 2) * coeff

            # 交叉项
            for h in range(l):
                z = n * N + h
                coeff = -2 * P * p_ij * (2 ** h)
                for k in range(N):
                    q = i * N + k
                    Q[q, z] += coeff * (0.5 - bits[k])

    # 3. C_max项
    for h in range(l):
        z = n * N + h
        Q[z, z] += 2 ** h + P * (2 ** (2 * h)) * n

    Q = (Q + Q.T) / 2
    return Q


def machine_indicator(start, bits):
    terms = {}
    for k in range(N):
        q = start + k
        terms[(q, q)] = 0.5 - bits[k]
        for j in range(k + 1, N):
            terms[(q, start + j)] = 0.25
    return terms


# ================== 验证哈密顿量 ==================
def validate_hamiltonian(Q):
    assert Q.shape == (num_qubits, num_qubits), "矩阵维度错误"
    assert np.allclose(Q, Q.T), "矩阵不对称"
    print("哈密顿量验证通过")


Q = build_hamiltonian()
validate_hamiltonian(Q)

# 转换为泡利算符
pauli_terms = []
for i in range(num_qubits):
    for j in range(i, num_qubits):
        if Q[i, j] != 0:
            pauli_str = ['I'] * num_qubits
            pauli_str[i] = 'Z'
            if j != i: pauli_str[j] = 'Z'
            pauli_terms.append((''.join(pauli_str), Q[i, j]))
hamiltonian = PauliSumOp.from_list(pauli_terms)


# ================== 优化算法 ==================
class QAOAOptimizer:
    def __init__(self, hamiltonian, max_layers, trials):
        self.hamiltonian = hamiltonian
        self.max_layers = max_layers
        self.trials = trials
        self.optimal = {'params': [], 'energy': float('inf')}
        self.history = []

    def optimize(self):
        backend = Aer.get_backend('qasm_simulator')

        for layer in range(1, self.max_layers + 1):
            print(f"Optimizing layer {layer}")
            # 参数维度验证
            expected_params = 2 * layer
            # 生成初始参数
            params_set = self._generate_params(layer)

            for params in params_set:
                if len(params) != expected_params:
                    raise ValueError(
                        f"参数维度错误！当前层数需要{expected_params}参数，实际得到{len(params)}个参数"
                    )
                qaoa = QAOA(optimizer=COBYLA(maxiter=1000),
                            reps=layer,
                            quantum_instance=backend,
                            initial_point=params)
                # print("layer: %d, params: %s" % (layer, params))
                result = qaoa.compute_minimum_eigenvalue(self.hamiltonian)
                energy = result.eigenvalue.real

                if energy < self.optimal['energy']:
                    self.optimal.update({
                        'params': list(result.optimal_parameters.values()),
                        'energy': energy
                    })
                    print(f"  New optimal energy: {energy:.4f}")

            # 早停检查
            # if len(self.history) > 1 and \
            #    abs(self.history[-1] - self.history[-2]) < 0.01:
            #     break
            # self.history.append(self.optimal['energy'])

        return self.optimal

    def _generate_params(self, layer):
        if layer == 1:
            return [np.random.uniform(0, 2 * np.pi, 2) for _ in range(self.trials)]

        # 扩展参数
        params_set = []
        base = self.optimal['params']
        for _ in range(self.trials):
            # 确保基础参数长度 = 2*(layer-1)
            if len(base) != 2 * (layer - 1):
                base = list(base) + list(np.random.uniform(0, 2 * np.pi, 2 * (layer - 1) - len(base)))
            # 添加新参数 γ_{layer-1}, β_{layer-1}
            new_params = list(base) + list(np.random.uniform(0, 2 * np.pi, 2))
            params_set.append(new_params)
        return params_set


# ================== 改进的结果解析 ==================
def analyze_result(result):
    from qiskit.visualization import plot_histogram

    # 获取测量结果
    counts = result.eigenstate
    print("\n测量结果分布:")
    display(plot_histogram(counts))  # Jupyter环境直接显示

    # 解析最优解
    optimal_bitstr = max(counts, key=lambda k: counts[k])
    schedule = decode_schedule(optimal_bitstr)

    print("\n最优调度方案:")
    print(f"C_max值: {schedule['c_max']}")
    print(f"作业分配: {schedule['assignments']}")
    print(f"有效性: {'有效' if schedule['valid'] else '存在冲突'}")


def decode_schedule(bitstr):
    # 解析机器分配
    assignments = {}
    for i in range(n):
        bits = bitstr[i * N: (i + 1) * N]
        machine_id = int(bits, 2)
        assignments[f'Job{i}'] = f'M{machine_id}' if machine_id < m else '非法'

    # 解析C_max
    c_max = sum(int(bitstr[n * N + h]) * (2 ** h) for h in range(l))

    # 验证时间约束
    machine_times = {f'M{j}': 0 for j in range(m)}
    valid = True
    for i in range(n):
        machine = assignments[f'Job{i}'][1]
        if machine.isdigit():
            mid = int(machine)
            if mid >= m:
                valid = False
                break
            machine_times[f'M{mid}'] += p[i][mid]
    valid &= all(t <= c_max for t in machine_times.values())

    return {'c_max': c_max, 'assignments': assignments, 'valid': valid}


def decode_state(bitstr):
    assignments = {}
    c_max = 0
    # 解析作业分配
    for i in range(n):
        bits = [int(bitstr[i * N + k]) for k in range(N)]
        machine = int(''.join(map(str, bits)), 2)
        assignments[f'Job{i}'] = f'M{machine}' if machine < m else '非法'
    # 解析C_max
    for h in range(l):
        c_max += int(bitstr[n * N + h]) * (2 ** h)
    # 验证约束
    machine_times = {f'M{j}': 0 for j in range(m)}
    for i in range(n):
        machine = assignments[f'Job{i}'][1]
        if machine.isdigit():
            mid = int(machine)
            machine_times[f'M{mid}'] += p[i][mid]
    valid = all(t <= c_max for t in machine_times.values())
    return {'c_max': c_max, 'assignments': assignments, 'valid': valid}


# ================== 主程序 ==================
# 执行优化
# 主程序流程
# if __name__ == "__main__":
# 构建哈密顿量
# H = build_hamiltonian()

# 执行QAOA优化
# optimizer = QAOAOptimizer(H, max_layers=3, trials=5)
# result = optimizer.optimize()

# 分析结果
# analyze_result(result)

# 验证最优解
# optimal_qc = build_qaoa_circuit(H, result['params'])
# counts = execute(optimal_qc, Aer.get_backend('qasm_simulator'), shots=1000).result().get_counts()
# print("\n最终测量结果:", sorted(counts.items(), key=lambda x: -x[1])[:3])
optimizer = QAOAOptimizer(hamiltonian, max_layers=5, trials=30)
result = optimizer.optimize()
print(result)
# 获取最终量子态
# qaoa = QAOA(reps=len(result['params'])//2,
#            optimizer=COBYLA(maxiter=0),
#            initial_point=result['params'])
# final_result = qaoa.compute_minimum_eigenvalue(hamiltonian)
# optimizer = COBYLA(maxiter=1000)


哈密顿量验证通过
Optimizing layer 1
  New optimal energy: -2514.4434
  New optimal energy: -5975.8477
  New optimal energy: -6834.4414
  New optimal energy: -7168.2969
  New optimal energy: -8194.2500
  New optimal energy: -14686.9531
  New optimal energy: -17840.8613
Optimizing layer 2
  New optimal energy: -21662.0645
  New optimal energy: -26759.2441
Optimizing layer 3
Optimizing layer 4
Optimizing layer 5
  New optimal energy: -31939.1113
{'params': [4.056590556349405, 3.820655497115159, 6.263703821417543, 3.210944980753072, 4.442746992257863, 2.2469702424973024, 2.8887737337931774, 1.1457068606822016, 6.02910705218638, 6.104943763964382], 'energy': -31939.111328125}


In [19]:
from qiskit.circuit import Parameter
from qiskit.opflow import PauliTrotterEvolution, Suzuki


def build_qaoa_circuit(hamiltonian, num_layers=1):
    """
    构建QAOA量子线路

    参数:
        hamiltonian (PauliSumOp): 问题哈密顿量
        num_layers (int): QAOA的层数（默认1层）

    返回:
        QuantumCircuit: 构造好的量子线路
    """
    num_qubits = hamiltonian.num_qubits
    qc = QuantumCircuit(num_qubits)

    # === 初始化 ===
    qc.h(range(num_qubits))  # 制备叠加态

    # === 定义参数 ===
    gammas = [Parameter(f'γ{i}') for i in range(num_layers)]
    betas = [Parameter(f'β{i}') for i in range(num_layers)]

    # === 分块演化 ===
    for layer in range(num_layers):
        # 问题哈密顿量演化: e^{-iγ H_P}
        problem_hamil = gammas[layer] * hamiltonian
        evolution = PauliTrotterEvolution(trotter_mode=Suzuki(order=1)).convert(problem_hamil.exp_i())
        qc.append(evolution.to_circuit(), qc.qubits)

        # 混合哈密顿量演化: e^{-iβ ΣX_i}
        mixer = sum(PauliSumOp.from_list([('X' * i + 'I' * (num_qubits - 1), 1)]) for i in range(num_qubits))
        mixer_hamil = betas[layer] * mixer
        mixer_evolution = PauliTrotterEvolution(trotter_mode=Suzuki(order=1)).convert(mixer_hamil.exp_i())
        qc.append(mixer_evolution.to_circuit(), qc.qubits)

    # === 测量 ===
    qc.measure_all()
    return qc


qc = build_qaoa_circuit(hamiltonian, num_layers=2)
# 查看线路结构
print(qc.draw())

ValueError: Sum of operators with different numbers of qubits, 5 and 6, is not well defined

In [17]:
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt
import matplotlib

matplotlib.use('TkAgg')

quantum_instance = Aer.get_backend('qasm_simulator', shots=10000)

qaoa = QAOA(
    optimizer=COBYLA(maxiter=10000),
    reps=5,
    quantum_instance=quantum_instance,
    initial_point=result['params']  # 初始参数
)
final_result = qaoa.compute_minimum_eigenvalue(hamiltonian)
counts = final_result.eigenstate
sorted_dict = {item[0][::-1]: item[1] for item in sorted(counts.items(), key=lambda item: item[1], reverse=True)}
filtered_dict = {key: value for key, value in sorted_dict.items() if value >= 0.1}
print(filtered_dict)
# plot_histogram(sorted_dict)
fig = plot_histogram(filtered_dict)
plt.show()
# 分析结果

analyze_result(final_result)
print(f"最低能量值: {result['energy']:.4f}")
print(f"最优参数: {result['params']}")

{'110110': 0.3801726581436387, '010110': 0.32923917977664807, '010111': 0.3125, '000110': 0.22534695471649932, '110111': 0.21650635094610965, '100110': 0.2096313728906053, '001010': 0.20252314682524564, '011110': 0.20009763241977652, '000100': 0.19008632907181935, '100101': 0.184877493221863, '000111': 0.1795175827043134, '000101': 0.15934435979977452, '001011': 0.15934435979977452, '011111': 0.15625, '111110': 0.14657549249448218, '011010': 0.13621559198564606, '100111': 0.12103072956898178, '110010': 0.11267347735824966, '001111': 0.11267347735824966, '010010': 0.10825317547305482}

测量结果分布:


<Figure size 700x500 with 1 Axes>


最优调度方案:
C_max值: 3
作业分配: {'Job0': 'M1', 'Job1': 'M2'}
有效性: 有效
最低能量值: -31939.1113
最优参数: [4.056590556349405, 3.820655497115159, 6.263703821417543, 3.210944980753072, 4.442746992257863, 2.2469702424973024, 2.8887737337931774, 1.1457068606822016, 6.02910705218638, 6.104943763964382]
